# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
len(df)

500

In [14]:
import re
from datetime import datetime, timezone


#replace month name with numerical value
def alph_to_digit(month):
    month_cld = ''
    if re.search(r'\bJan\w*',month,flags=re.I):
        month_cld = re.search(r'\bJan\w*',month,flags=re.I).group()
        return month.replace(month_cld,'1')
    elif re.search(r'\bfeb\w*',month,flags=re.I):
        month_cld = re.search(r'\bfeb\w*',month,flags=re.I).group()
        return month.replace(month_cld,'2')
    elif re.search(r'\bmar\w*',month,flags=re.I):
        month_cld = re.search(r'\bmar\w*',month,flags=re.I).group()
        return month.replace(month_cld,'3')
    elif re.search(r'\bapr\w*',month,flags=re.I):
        month_cld = re.search(r'\bapr\w*',month,flags=re.I).group()
        return month.replace(month_cld,'4')
    elif re.search(r'\bmay\w*',month,flags=re.I):
        month_cld = re.search(r'\bmay\w*',month,flags=re.I).group()
        return month.replace(month_cld,'5')
    elif re.search(r'\bjun\w*',month,flags=re.I):
        month_cld = re.search(r'\bjun\w*',month,flags=re.I).group()
        return month.replace(month_cld,'6')
    elif re.search(r'\bjul\w*',month,flags=re.I):
        month_cld = re.search(r'\bjul\w*',month,flags=re.I).group()
        return month.replace(month_cld,'7')
    elif re.search(r'\baug\w*',month,flags=re.I):
        month_cld = re.search(r'\baug\w*',month,flags=re.I).group()
        return month.replace(month_cld,'8')
    elif re.search(r'\bsep\w*',month,flags=re.I):
        month_cld = re.search(r'\bsep\w*',month,flags=re.I).group()
        return month.replace(month_cld,'9')
    elif re.search(r'\boct\w*',month,flags=re.I):
        month_cld = re.search(r'\boct\w*',month,flags=re.I).group()
        return month.replace(month_cld,'10')
    elif re.search(r'\bnov\w*',month,flags=re.I):
        month_cld = re.search(r'\bnov\w*',month,flags=re.I).group()
        return month.replace(month_cld,'11')
    elif re.search(r'\bdec\w*',month,flags=re.I):
        month_cld = re.search(r'\bdec\w*',month,flags=re.I).group()
        return month.replace(month_cld,'12')
    else:
        return month
    

    
def date_sorter():
    data = pd.Series(doc)

    data_df = pd.DataFrame(data, columns=['text'])
    data_df['extracted date'] = ''

    m = data_df.shape[0]
    
    
    for m in range(m):
        txt_m = data_df.loc[m,'text']
        ##extract variations of date one seach for each variation of date format
        # 04/20/2009; 04/20/09; 4/20/09; 4/3/09
        if (re.search('\d{1,2}[/-]\d{1,2}[/-]\d{4}', txt_m)) is not None:
            ext = (re.search('\d{1,2}[/-]\d{1,2}[/-]\d{4}', txt_m)).group()
            ext = ext.replace('-', '/')
            ext = datetime.strptime(ext, '%m/%d/%Y')
            data_df.loc[m, 'extracted date'] = ext
            continue

        if (re.search('\d{1,2}[/-]\d{1,2}[/-]\d{2}', txt_m)) is not None:
            ext = (re.search('\d{1,2}[/-]\d{1,2}[/-]\d{2}', txt_m)).group()
            ext = ext.replace('-', '/')
            ext = ext[:-2] + '19' + ext[-2:]
            ext = datetime.strptime(ext, "%m/%d/%Y")
            data_df.loc[m, 'extracted date'] = ext
            continue

        # Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
        if (re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-zA-Z.,-]*[,\s-](\d{1,2})[,\s-]+\d{4}', txt_m))is not None:
            ext = (re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-zA-Z.,-]*[,\s-](\d{1,2})[,\s-]+\d{4}', txt_m)).group()
            #convert to the right format
            ext = ext.replace(' ', '/').replace(',', '').replace('.', '').replace('-', '/')
            ext = alph_to_digit(ext)
            ext = datetime.strptime(ext, "%m/%d/%Y")
            data_df.loc[m, 'extracted date'] = ext
            continue

        # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
        if (re.search(r'(\d{1,2})[,\s-](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-zA-Z.,-]*[\s]\d{4}', txt_m)) is not None:
            ext = (re.search(r'(\d{1,2})[,\s-](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-zA-Z.,-]*[\s]\d{4}', txt_m)).group()
            #correct the format
            ext = ext.replace(' ', '/').replace(',', '').replace('.', '').replace('-', '/')
            ext = alph_to_digit(ext)
            ext = datetime.strptime(ext, "%d/%m/%Y")
            data_df.loc[m, 'extracted date'] = ext
            continue

        #Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
        if (re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-zA-Z.,-]*[,\s-](\d{1,2})\w{2}[,\s-]\d{4}', txt_m))is not None:
            ext = (re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-zA-Z.,-]*[,\s-](\d{1,2})\w{2}[,\s-]\d{4}', txt_m)).group()
            #convert to the right format
            ext = ext.replace(' ', '/').replace(',', '').replace('-', '/')
            ext = alph_to_digit(ext)
            ext = datetime.strptime(ext, "%m/%d/%Y")
            data_df.loc[m, 'extracted date'] = ext
            continue

        #Feb 2009; Sep 2009; Oct 2010
        if (re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-zA-Z.,-]*[,\s-]+\d{4}', txt_m))is not None:
            ext = (re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-zA-Z.,-]*[,\s-]+\d{4}', txt_m)).group()
            #convert to the right format
            ext = ext.replace(' ', '/').replace(',', '').replace('.', '').replace('-', '/')
            ext = alph_to_digit(ext)
            ext = datetime.strptime(ext, "%m/%Y")
            data_df.loc[m, 'extracted date'] = ext
            continue


        # 6/2008; 12/2009
        if (re.search(r'\d{1,2}[/]\d{4}', txt_m)) is not None:
            ext = (re.search(r'\d{1,2}[/-]\d{4}', txt_m)).group()
            #add day - convert to the right format - and add it to the df
            ext = datetime.strptime(ext, "%m/%Y")
            data_df.loc[m, 'extracted date'] = ext
            continue

        # 2009; 2010
        if (re.search(r'\d{4}', txt_m)) is not None:
            ext = (re.search(r'\d{4}', txt_m)).group()
            #add day - convert to the right format - and add it to the df
            ext = datetime.strptime(ext, "%Y")
            data_df.loc[m, 'extracted date'] = ext

        
    data_df.reset_index(inplace = True)
    data_df.rename(columns = {"index" : "original row number"}, inplace= True)


    data_df = data_df.sort_values(by=['extracted date', 'original row number'])

    sorted_idx = data_df.index.values.tolist()


    new_series = pd.Series(sorted_idx)

    return (new_series)



date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64